In [2]:
from bs4 import BeautifulSoup as soup #parses HTML
from urllib.request import urlopen as uReq #opens webpage
import spacy
import time
import json
import pandas as pd
from Levenshtein import distance as lev

In [3]:
#all of these lists are hard coded to improve runtime. These are all from the constituents.csv file int he github repository and represent the S&P 500 as of 11/27/2023
#if you want to expand functionality, you would either make these lists at runtime or just add the new stocks to the runtime
tickers = ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADM', 'ADBE', 'ADP', 'AES', 'AFL', 'A', 'ABNB', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV', 'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EG', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX', 'FITB', 'FSLR', 'FE', 'FIS', 'FI', 'FLT', 'FMC', 'F', 'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GEHC', 'GEN', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUBB', 'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'ITW', 'ILMN', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG', 'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KVUE', 'KDP', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LULU', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OTIS', 'PCAR', 'PKG', 'PANW', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC', 'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RVTY', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SNA', 'SEDG', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UDR', 'ULTA', 'UNP', 'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VLTO', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VFC', 'VTRS', 'VICI', 'V', 'VMC', 'WAB', 'WBA', 'WMT', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WRK', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']
industries = ['Industrials', 'Industrials', 'Health Care', 'Health Care', 'Information Technology', 'Consumer Staples', 'Information Technology', 'Industrials', 'Utilities', 'Financials', 'Health Care', 'Consumer Discretionary', 'Materials', 'Information Technology', 'Industrials', 'Materials', 'Real Estate', 'Health Care', 'Industrials', 'Utilities', 'Financials', 'Communication Services', 'Communication Services', 'Consumer Staples', 'Consumer Discretionary', 'Materials', 'Information Technology', 'Utilities', 'Industrials', 'Utilities', 'Financials', 'Financials', 'Real Estate', 'Utilities', 'Financials', 'Industrials', 'Health Care', 'Information Technology', 'Information Technology', 'Information Technology', 'Financials', 'Energy', 'Information Technology', 'Information Technology', 'Consumer Discretionary', 'Financials', 'Information Technology', 'Financials', 'Financials', 'Communication Services', 'Utilities', 'Information Technology', 'Consumer Discretionary', 'Real Estate', 'Materials', 'Industrials', 'Energy', 'Materials', 'Financials', 'Consumer Discretionary', 'Health Care', 'Health Care', 'Financials', 'Financials', 'Consumer Discretionary', 'Health Care', 'Health Care', 'Health Care', 'Financials', 'Financials', 'Financials', 'Industrials', 'Consumer Discretionary', 'Consumer Discretionary', 'Real Estate', 'Health Care', 'Health Care', 'Information Technology', 'Industrials', 'Financials', 'Consumer Staples', 'Consumer Staples', 'Industrials', 'Information Technology', 'Consumer Discretionary', 'Real Estate', 'Consumer Staples', 'Financials', 'Health Care', 'Consumer Discretionary', 'Consumer Discretionary', 'Industrials', 'Health Care', 'Industrials', 'Financials', 'Real Estate', 'Information Technology', 'Materials', 'Health Care', 'Health Care', 'Utilities', 'Industrials', 'Materials', 'Health Care', 'Financials', 'Communication Services', 'Energy', 'Consumer Discretionary', 'Financials', 'Consumer Staples', 'Health Care', 'Financials', 'Industrials', 'Information Technology', 'Financials', 'Financials', 'Consumer Staples', 'Financials', 'Utilities', 'Consumer Staples', 'Information Technology', 'Consumer Staples', 'Communication Services', 'Financials', 'Consumer Staples', 'Energy', 'Utilities', 'Consumer Staples', 'Utilities', 'Health Care', 'Industrials', 'Information Technology', 'Materials', 'Real Estate', 'Consumer Staples', 'Energy', 'Real Estate', 'Industrials', 'Industrials', 'Health Care', 'Consumer Discretionary', 'Health Care', 'Consumer Discretionary', 'Health Care', 'Industrials', 'Industrials', 'Health Care', 'Energy', 'Health Care', 'Energy', 'Real Estate', 'Financials', 'Communication Services', 'Consumer Staples', 'Consumer Staples', 'Utilities', 'Consumer Discretionary', 'Industrials', 'Materials', 'Utilities', 'Utilities', 'Materials', 'Materials', 'Industrials', 'Consumer Discretionary', 'Materials', 'Utilities', 'Health Care', 'Communication Services', 'Health Care', 'Health Care', 'Industrials', 'Information Technology', 'Utilities', 'Energy', 'Information Technology', 'Energy', 'Industrials', 'Real Estate', 'Real Estate', 'Real Estate', 'Consumer Staples', 'Consumer Discretionary', 'Financials', 'Utilities', 'Utilities', 'Utilities', 'Consumer Discretionary', 'Industrials', 'Real Estate', 'Energy', 'Information Technology', 'Financials', 'Information Technology', 'Industrials', 'Real Estate', 'Industrials', 'Financials', 'Information Technology', 'Utilities', 'Financials', 'Financials', 'Financials', 'Materials', 'Consumer Discretionary', 'Information Technology', 'Industrials', 'Communication Services', 'Communication Services', 'Financials', 'Materials', 'Consumer Discretionary', 'Information Technology', 'Health Care', 'Information Technology', 'Industrials', 'Industrials', 'Industrials', 'Consumer Staples', 'Consumer Discretionary', 'Consumer Discretionary', 'Health Care', 'Financials', 'Financials', 'Financials', 'Energy', 'Financials', 'Consumer Discretionary', 'Health Care', 'Real Estate', 'Health Care', 'Consumer Staples', 'Energy', 'Information Technology', 'Consumer Discretionary', 'Health Care', 'Consumer Discretionary', 'Industrials', 'Consumer Staples', 'Real Estate', 'Industrials', 'Information Technology', 'Industrials', 'Health Care', 'Financials', 'Industrials', 'Information Technology', 'Industrials', 'Health Care', 'Industrials', 'Health Care', 'Health Care', 'Industrials', 'Health Care', 'Information Technology', 'Financials', 'Materials', 'Materials', 'Communication Services', 'Information Technology', 'Health Care', 'Financials', 'Real Estate', 'Health Care', 'Real Estate', 'Industrials', 'Financials', 'Industrials', 'Health Care', 'Industrials', 'Financials', 'Information Technology', 'Consumer Staples', 'Consumer Staples', 'Consumer Staples', 'Financials', 'Information Technology', 'Consumer Staples', 'Real Estate', 'Energy', 'Information Technology', 'Consumer Staples', 'Consumer Staples', 'Industrials', 'Health Care', 'Information Technology', 'Consumer Staples', 'Consumer Discretionary', 'Industrials', 'Consumer Discretionary', 'Materials', 'Communication Services', 'Consumer Discretionary', 'Industrials', 'Financials', 'Consumer Discretionary', 'Consumer Discretionary', 'Materials', 'Financials', 'Energy', 'Energy', 'Financials', 'Consumer Discretionary', 'Financials', 'Materials', 'Industrials', 'Financials', 'Communication Services', 'Consumer Staples', 'Consumer Discretionary', 'Health Care', 'Health Care', 'Health Care', 'Communication Services', 'Financials', 'Health Care', 'Consumer Discretionary', 'Information Technology', 'Information Technology', 'Information Technology', 'Real Estate', 'Health Care', 'Consumer Discretionary', 'Health Care', 'Consumer Staples', 'Consumer Staples', 'Information Technology', 'Consumer Staples', 'Financials', 'Financials', 'Materials', 'Information Technology', 'Financials', 'Financials', 'Information Technology', 'Communication Services', 'Materials', 'Communication Services', 'Communication Services', 'Utilities', 'Consumer Discretionary', 'Utilities', 'Industrials', 'Industrials', 'Financials', 'Industrials', 'Consumer Discretionary', 'Utilities', 'Materials', 'Information Technology', 'Consumer Discretionary', 'Information Technology', 'Consumer Discretionary', 'Energy', 'Industrials', 'Communication Services', 'Information Technology', 'Energy', 'Information Technology', 'Industrials', 'Industrials', 'Materials', 'Information Technology', 'Communication Services', 'Industrials', 'Industrials', 'Industrials', 'Financials', 'Industrials', 'Consumer Staples', 'Health Care', 'Utilities', 'Consumer Staples', 'Energy', 'Utilities', 'Energy', 'Financials', 'Consumer Discretionary', 'Materials', 'Utilities', 'Financials', 'Consumer Staples', 'Financials', 'Real Estate', 'Financials', 'Utilities', 'Information Technology', 'Real Estate', 'Consumer Discretionary', 'Information Technology', 'Industrials', 'Information Technology', 'Health Care', 'Consumer Discretionary', 'Financials', 'Industrials', 'Real Estate', 'Real Estate', 'Health Care', 'Financials', 'Industrials', 'Health Care', 'Health Care', 'Industrials', 'Industrials', 'Industrials', 'Information Technology', 'Consumer Discretionary', 'Consumer Discretionary', 'Financials', 'Information Technology', 'Real Estate', 'Energy', 'Information Technology', 'Materials', 'Utilities', 'Information Technology', 'Materials', 'Real Estate', 'Information Technology', 'Consumer Staples', 'Industrials', 'Information Technology', 'Utilities', 'Industrials', 'Industrials', 'Consumer Discretionary', 'Financials', 'Materials', 'Health Care', 'Health Care', 'Financials', 'Information Technology', 'Consumer Staples', 'Communication Services', 'Financials', 'Communication Services', 'Consumer Discretionary', 'Energy', 'Consumer Staples', 'Information Technology', 'Information Technology', 'Health Care', 'Information Technology', 'Consumer Discretionary', 'Information Technology', 'Industrials', 'Health Care', 'Consumer Discretionary', 'Consumer Discretionary', 'Industrials', 'Industrials', 'Financials', 'Information Technology', 'Financials', 'Information Technology', 'Consumer Staples', 'Financials', 'Real Estate', 'Consumer Discretionary', 'Industrials', 'Industrials', 'Industrials', 'Industrials', 'Health Care', 'Health Care', 'Energy', 'Real Estate', 'Industrials', 'Information Technology', 'Industrials', 'Communication Services', 'Health Care', 'Consumer Discretionary', 'Health Care', 'Real Estate', 'Financials', 'Materials', 'Industrials', 'Consumer Staples', 'Consumer Staples', 'Communication Services', 'Industrials', 'Health Care', 'Utilities', 'Financials', 'Real Estate', 'Health Care', 'Information Technology', 'Materials', 'Real Estate', 'Consumer Discretionary', 'Energy', 'Financials', 'Industrials', 'Consumer Discretionary', 'Utilities', 'Industrials', 'Consumer Discretionary', 'Information Technology', 'Health Care', 'Financials', 'Health Care']
subindustries = ['Industrial Conglomerates', 'Building Products', 'Health Care Equipment', 'Pharmaceuticals', 'IT Consulting & Other Services', 'Agricultural Products & Services', 'Application Software', 'Human Resource & Employment Services', 'Independent Power Producers & Energy Traders', 'Life & Health Insurance', 'Health Care Equipment', 'Hotels, Resorts & Cruise Lines', 'Industrial Gases', 'Internet Services & Infrastructure', 'Passenger Airlines', 'Specialty Chemicals', 'Office REITs', 'Health Care Supplies', 'Building Products', 'Electric Utilities', 'Property & Casualty Insurance', 'Interactive Media & Services', 'Interactive Media & Services', 'Tobacco', 'Broadline Retail', 'Paper & Plastic Packaging Products & Materials', 'Semiconductors', 'Multi-Utilities', 'Passenger Airlines', 'Electric Utilities', 'Consumer Finance', 'Property & Casualty Insurance', 'Telecom Tower REITs', 'Water Utilities', 'Asset Management & Custody Banks', 'Electrical Components & Equipment', 'Biotechnology', 'Electronic Components', 'Semiconductors', 'Application Software', 'Insurance Brokers', 'Oil & Gas Exploration & Production', 'Technology Hardware, Storage & Peripherals', 'Semiconductor Materials & Equipment', 'Automotive Parts & Equipment', 'Reinsurance', 'Communications Equipment', 'Insurance Brokers', 'Multi-line Insurance', 'Integrated Telecommunication Services', 'Gas Utilities', 'Application Software', 'Automotive Retail', 'Multi-Family Residential REITs', 'Paper & Plastic Packaging Products & Materials', 'Aerospace & Defense', 'Oil & Gas Equipment & Services', 'Metal, Glass & Plastic Containers', 'Diversified Banks', 'Specialty Stores', 'Health Care Equipment', 'Health Care Equipment', 'Property & Casualty Insurance', 'Multi-Sector Holdings', 'Computer & Electronics Retail', 'Life Sciences Tools & Services', 'Life Sciences Tools & Services', 'Biotechnology', 'Asset Management & Custody Banks', 'Asset Management & Custody Banks', 'Asset Management & Custody Banks', 'Aerospace & Defense', 'Hotels, Resorts & Cruise Lines', 'Automotive Parts & Equipment', 'Office REITs', 'Health Care Equipment', 'Pharmaceuticals', 'Semiconductors', 'Data Processing & Outsourced Services', 'Insurance Brokers', 'Distillers & Vintners', 'Agricultural Products & Services', 'Air Freight & Logistics', 'Application Software', 'Casinos & Gaming', 'Multi-Family Residential REITs', 'Packaged Foods & Meats', 'Consumer Finance', 'Health Care Distributors', 'Automotive Retail', 'Hotels, Resorts & Cruise Lines', 'Building Products', 'Pharmaceuticals', 'Construction Machinery & Heavy Transportation Equipment', 'Financial Exchanges & Data', 'Real Estate Services', 'Technology Distributors', 'Specialty Chemicals', 'Health Care Distributors', 'Managed Health Care', 'Multi-Utilities', 'Human Resource & Employment Services', 'Fertilizers & Agricultural Chemicals', 'Life Sciences Tools & Services', 'Investment Banking & Brokerage', 'Cable & Satellite', 'Integrated Oil & Gas', 'Restaurants', 'Property & Casualty Insurance', 'Household Products', 'Managed Health Care', 'Property & Casualty Insurance', 'Diversified Support Services', 'Communications Equipment', 'Diversified Banks', 'Regional Banks', 'Household Products', 'Financial Exchanges & Data', 'Multi-Utilities', 'Soft Drinks & Non-alcoholic Beverages', 'IT Consulting & Other Services', 'Household Products', 'Cable & Satellite', 'Diversified Banks', 'Packaged Foods & Meats', 'Oil & Gas Exploration & Production', 'Electric Utilities', 'Distillers & Vintners', 'Multi-Utilities', 'Health Care Supplies', 'Diversified Support Services', 'Electronic Components', 'Fertilizers & Agricultural Chemicals', 'Real Estate Services', 'Consumer Staples Merchandise Retail', 'Oil & Gas Exploration & Production', 'Telecom Tower REITs', 'Rail Transportation', 'Industrial Machinery & Supplies & Components', 'Health Care Services', 'Homebuilding', 'Health Care Equipment', 'Restaurants', 'Health Care Facilities', 'Agricultural & Farm Machinery', 'Passenger Airlines', 'Health Care Supplies', 'Oil & Gas Exploration & Production', 'Health Care Equipment', 'Oil & Gas Exploration & Production', 'Data Center REITs', 'Consumer Finance', 'Movies & Entertainment', 'Consumer Staples Merchandise Retail', 'Consumer Staples Merchandise Retail', 'Electric Utilities', 'Restaurants', 'Industrial Machinery & Supplies & Components', 'Commodity Chemicals', 'Multi-Utilities', 'Electric Utilities', 'Specialty Chemicals', 'Diversified Chemicals', 'Electrical Components & Equipment', 'Broadline Retail', 'Specialty Chemicals', 'Electric Utilities', 'Health Care Equipment', 'Interactive Home Entertainment', 'Managed Health Care', 'Pharmaceuticals', 'Electrical Components & Equipment', 'Electronic Components', 'Electric Utilities', 'Oil & Gas Exploration & Production', 'IT Consulting & Other Services', 'Oil & Gas Exploration & Production', 'Research & Consulting Services', 'Data Center REITs', 'Multi-Family Residential REITs', 'Multi-Family Residential REITs', 'Personal Care Products', 'Broadline Retail', 'Reinsurance', 'Electric Utilities', 'Multi-Utilities', 'Multi-Utilities', 'Hotels, Resorts & Cruise Lines', 'Air Freight & Logistics', 'Self-Storage REITs', 'Integrated Oil & Gas', 'Communications Equipment', 'Financial Exchanges & Data', 'Application Software', 'Building Products', 'Retail REITs', 'Air Freight & Logistics', 'Regional Banks', 'Semiconductors', 'Electric Utilities', 'Transaction & Payment Processing Services', 'Transaction & Payment Processing Services', 'Transaction & Payment Processing Services', 'Fertilizers & Agricultural Chemicals', 'Automobile Manufacturers', 'Systems Software', 'Industrial Machinery & Supplies & Components', 'Movies & Entertainment', 'Movies & Entertainment', 'Asset Management & Custody Banks', 'Copper', 'Consumer Electronics', 'IT Consulting & Other Services', 'Health Care Technology', 'Application Software', 'Electrical Components & Equipment', 'Aerospace & Defense', 'Industrial Conglomerates', 'Packaged Foods & Meats', 'Automobile Manufacturers', 'Automotive Parts & Equipment', 'Biotechnology', 'Life & Health Insurance', 'Transaction & Payment Processing Services', 'Investment Banking & Brokerage', 'Oil & Gas Equipment & Services', 'Property & Casualty Insurance', 'Leisure Products', 'Health Care Facilities', 'Health Care REITs', 'Health Care Distributors', 'Packaged Foods & Meats', 'Integrated Oil & Gas', 'Technology Hardware, Storage & Peripherals', 'Hotels, Resorts & Cruise Lines', 'Health Care Equipment', 'Home Improvement Retail', 'Industrial Conglomerates', 'Packaged Foods & Meats', 'Hotel & Resort REITs', 'Aerospace & Defense', 'Technology Hardware, Storage & Peripherals', 'Industrial Machinery', 'Managed Health Care', 'Regional Banks', 'Aerospace & Defense', 'IT Consulting & Other Services', 'Industrial Machinery & Supplies & Components', 'Health Care Equipment', 'Industrial Machinery & Supplies & Components', 'Life Sciences Tools & Services', 'Biotechnology', 'Industrial Machinery & Supplies & Components', 'Health Care Equipment', 'Semiconductors', 'Financial Exchanges & Data', 'Specialty Chemicals', 'Paper & Plastic Packaging Products & Materials', 'Advertising', 'Application Software', 'Health Care Equipment', 'Asset Management & Custody Banks', 'Single-Family Residential REITs', 'Life Sciences Tools & Services', 'Other Specialized REITs', 'Cargo Ground Transportation', 'Transaction & Payment Processing Services', 'Construction & Engineering', 'Pharmaceuticals', 'Building Products', 'Diversified Banks', 'Communications Equipment', 'Packaged Foods & Meats', 'Personal Care Products', 'Soft Drinks & Non-alcoholic Beverages', 'Regional Banks', 'Electronic Equipment & Instruments', 'Household Products', 'Retail REITs', 'Oil & Gas Storage & Transportation', 'Semiconductor Materials & Equipment', 'Packaged Foods & Meats', 'Food Retail', 'Aerospace & Defense', 'Health Care Services', 'Semiconductor Materials & Equipment', 'Packaged Foods & Meats', 'Casinos & Gaming', 'Diversified Support Services', 'Homebuilding', 'Industrial Gases', 'Movies & Entertainment', 'Distributors', 'Aerospace & Defense', 'Multi-line Insurance', 'Home Improvement Retail', 'Apparel, Accessories & Luxury Goods', 'Specialty Chemicals', 'Regional Banks', 'Oil & Gas Exploration & Production', 'Oil & Gas Refining & Marketing', 'Financial Exchanges & Data', 'Hotels, Resorts & Cruise Lines', 'Insurance Brokers', 'Construction Materials', 'Building Products', 'Transaction & Payment Processing Services', 'Interactive Media & Services', 'Packaged Foods & Meats', 'Restaurants', 'Health Care Distributors', 'Health Care Equipment', 'Pharmaceuticals', 'Interactive Media & Services', 'Life & Health Insurance', 'Life Sciences Tools & Services', 'Casinos & Gaming', 'Semiconductors', 'Semiconductors', 'Systems Software', 'Multi-Family Residential REITs', 'Biotechnology', 'Home Furnishings', 'Managed Health Care', 'Brewers', 'Packaged Foods & Meats', 'Semiconductors', 'Soft Drinks & Non-alcoholic Beverages', 'Financial Exchanges & Data', 'Investment Banking & Brokerage', 'Fertilizers & Agricultural Chemicals', 'Communications Equipment', 'Financial Exchanges & Data', 'Financial Exchanges & Data', 'Technology Hardware, Storage & Peripherals', 'Movies & Entertainment', 'Gold', 'Publishing', 'Publishing', 'Multi-Utilities', 'Apparel, Accessories & Luxury Goods', 'Multi-Utilities', 'Industrial Machinery & Supplies & Components', 'Rail Transportation', 'Asset Management & Custody Banks', 'Aerospace & Defense', 'Hotels, Resorts & Cruise Lines', 'Independent Power Producers & Energy Traders', 'Steel', 'Semiconductors', 'Homebuilding', 'Semiconductors', 'Automotive Retail', 'Oil & Gas Exploration & Production', 'Cargo Ground Transportation', 'Advertising', 'Semiconductors', 'Oil & Gas Storage & Transportation', 'Application Software', 'Industrial Machinery & Supplies & Components', 'Construction Machinery & Heavy Transportation Equipment', 'Paper & Plastic Packaging Products & Materials', 'Application Software', 'Movies & Entertainment', 'Industrial Machinery & Supplies & Components', 'Human Resource & Employment Services', 'Human Resource & Employment Services', 'Transaction & Payment Processing Services', 'Industrial Machinery & Supplies & Components', 'Soft Drinks & Non-alcoholic Beverages', 'Pharmaceuticals', 'Multi-Utilities', 'Tobacco', 'Oil & Gas Refining & Marketing', 'Multi-Utilities', 'Oil & Gas Exploration & Production', 'Regional Banks', 'Distributors', 'Specialty Chemicals', 'Electric Utilities', 'Life & Health Insurance', 'Personal Care Products', 'Property & Casualty Insurance', 'Industrial REITs', 'Life & Health Insurance', 'Electric Utilities', 'Application Software', 'Self-Storage REITs', 'Homebuilding', 'Semiconductors', 'Construction & Engineering', 'Semiconductors', 'Health Care Services', 'Apparel, Accessories & Luxury Goods', 'Investment Banking & Brokerage', 'Aerospace & Defense', 'Retail REITs', 'Retail REITs', 'Biotechnology', 'Regional Banks', 'Environmental & Facilities Services', 'Health Care Equipment', 'Health Care Equipment', 'Human Resource & Employment Services', 'Electrical Components & Equipment', 'Environmental & Facilities Services', 'Electronic Equipment & Instruments', 'Apparel Retail', 'Hotels, Resorts & Cruise Lines', 'Financial Exchanges & Data', 'Application Software', 'Telecom Tower REITs', 'Oil & Gas Equipment & Services', 'Technology Hardware, Storage & Peripherals', 'Paper & Plastic Packaging Products & Materials', 'Multi-Utilities', 'Systems Software', 'Specialty Chemicals', 'Retail REITs', 'Semiconductors', 'Packaged Foods & Meats', 'Industrial Machinery & Supplies & Components', 'Semiconductor Materials & Equipment', 'Electric Utilities', 'Passenger Airlines', 'Industrial Machinery & Supplies & Components', 'Restaurants', 'Asset Management & Custody Banks', 'Steel', 'Health Care Equipment', 'Health Care Equipment', 'Consumer Finance', 'Application Software', 'Food Distributors', 'Wireless Telecommunication Services', 'Asset Management & Custody Banks', 'Interactive Home Entertainment', 'Apparel, Accessories & Luxury Goods', 'Oil & Gas Storage & Transportation', 'Consumer Staples Merchandise Retail', 'Electronic Manufacturing Services', 'Electronic Equipment & Instruments', 'Health Care Equipment', 'Semiconductor Materials & Equipment', 'Automobile Manufacturers', 'Semiconductors', 'Aerospace & Defense', 'Life Sciences Tools & Services', 'Apparel Retail', 'Specialty Stores', 'Building Products', 'Aerospace & Defense', 'Property & Casualty Insurance', 'Electronic Equipment & Instruments', 'Regional Banks', 'Application Software', 'Packaged Foods & Meats', 'Diversified Banks', 'Multi-Family Residential REITs', 'Specialty Stores', 'Rail Transportation', 'Passenger Airlines', 'Air Freight & Logistics', 'Trading Companies & Distributors', 'Managed Health Care', 'Health Care Facilities', 'Oil & Gas Refining & Marketing', 'Health Care REITs', 'Environmental & Facilities Services', 'Internet Services & Infrastructure', 'Research & Consulting Services', 'Integrated Telecommunication Services', 'Biotechnology', 'Apparel, Accessories & Luxury Goods', 'Pharmaceuticals', 'Hotel & Resort REITs', 'Transaction & Payment Processing Services', 'Construction Materials', 'Construction Machinery & Heavy Transportation Equipment', 'Drug Retail', 'Consumer Staples Merchandise Retail', 'Broadcasting', 'Environmental & Facilities Services', 'Health Care Distributors', 'Electric Utilities', 'Diversified Banks', 'Health Care REITs', 'Health Care Supplies', 'Technology Hardware, Storage & Peripherals', 'Paper & Plastic Packaging Products & Materials', 'Timber REITs', 'Household Appliances', 'Oil & Gas Storage & Transportation', 'Insurance Brokers', 'Industrial Machinery & Supplies & Components', 'Casinos & Gaming', 'Multi-Utilities', 'Industrial Machinery & Supplies & Components', 'Restaurants', 'Electronic Equipment & Instruments', 'Health Care Equipment', 'Regional Banks', 'Pharmaceuticals']
names = ['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'ADM', 'Adobe Inc.', 'ADP', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Airbnb', 'Air Products and Chemicals', 'Akamai', 'Alaska Air Group', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'AMD', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank of America', 'Bath & Body Works, Inc.', 'Baxter International', 'Becton Dickinson', 'Berkley', 'Berkshire Hathaway', 'Best Buy', 'Bio-Rad', 'Bio-Techne', 'Biogen', 'BlackRock', 'Blackstone', 'BNY Mellon', 'Boeing', 'Booking Holdings', 'BorgWarner', 'Boston Properties', 'Boston Scientific', 'Bristol Myers Squibb', 'Broadcom Inc.', 'Broadridge Financial Solutions', 'Brown & Brown', 'Brown–Forman', 'Bunge Limited', 'C.H. Robinson', 'Cadence Design Systems', 'Caesars Entertainment', 'Camden Property Trust', 'Campbell Soup Company', 'Capital One', 'Cardinal Health', 'CarMax', 'Carnival', 'Carrier Global', 'Catalent', 'Caterpillar Inc.', 'Cboe Global Markets', 'CBRE Group', 'CDW', 'Celanese', 'Cencora', 'Centene Corporation', 'CenterPoint Energy', 'Ceridian', 'CF Industries', 'Charles River Laboratories', 'Charles Schwab Corporation', 'Charter Communications', 'Chevron Corporation', 'Chipotle Mexican Grill', 'Chubb Limited', 'Church & Dwight', 'Cigna', 'Cincinnati Financial', 'Cintas', 'Cisco', 'Citigroup', 'Citizens Financial Group', 'Clorox', 'CME Group', 'CMS Energy', 'Coca-Cola Company (The)', 'Cognizant', 'Colgate-Palmolive', 'Comcast', 'Comerica', 'Conagra Brands', 'ConocoPhillips', 'Consolidated Edison', 'Constellation Brands', 'Constellation Energy', 'CooperCompanies', 'Copart', 'Corning Inc.', 'Corteva', 'CoStar Group', 'Costco', 'Coterra', 'Crown Castle', 'CSX', 'Cummins', 'CVS Health', 'D.R. Horton', 'Danaher Corporation', 'Darden Restaurants', 'DaVita Inc.', 'John Deere', 'Delta Air Lines', 'Dentsply Sirona', 'Devon Energy', 'Dexcom', 'Diamondback Energy', 'Digital Realty', 'Discover Financial', 'Disney', 'Dollar General', 'Dollar Tree', 'Dominion Energy', "Domino's", 'Dover Corporation', 'Dow Inc.', 'DTE Energy', 'Duke Energy', 'DuPont', 'Eastman Chemical Company', 'Eaton Corporation', 'eBay', 'Ecolab', 'Edison International', 'Edwards Lifesciences', 'Electronic Arts', 'Elevance Health', 'Eli Lilly and Company', 'Emerson Electric', 'Enphase', 'Entergy', 'EOG Resources', 'EPAM Systems', 'EQT', 'Equifax', 'Equinix', 'Equity Residential', 'Essex Property Trust', 'Estée Lauder Companies (The)', 'Etsy', 'Everest Re', 'Evergy', 'Eversource', 'Exelon', 'Expedia Group', 'Expeditors International', 'Extra Space Storage', 'ExxonMobil', 'F5, Inc.', 'FactSet', 'Fair Isaac', 'Fastenal', 'Federal Realty', 'FedEx', 'Fifth Third Bank', 'First Solar', 'FirstEnergy', 'FIS', 'Fiserv', 'Fleetcor', 'FMC Corporation', 'Ford Motor Company', 'Fortinet', 'Fortive', 'Fox Corporation (Class A)', 'Fox Corporation (Class B)', 'Franklin Templeton', 'Freeport-McMoRan', 'Garmin', 'Gartner', 'GE HealthCare', 'Gen Digital', 'Generac', 'General Dynamics', 'General Electric', 'General Mills', 'General Motors', 'Genuine Parts Company', 'Gilead Sciences', 'Globe Life', 'Global Payments', 'Goldman Sachs', 'Halliburton', 'Hartford (The)', 'Hasbro', 'HCA Healthcare', 'Healthpeak', 'Henry Schein', "Hershey's", 'Hess Corporation', 'Hewlett Packard Enterprise', 'Hilton Worldwide', 'Hologic', 'Home Depot (The)', 'Honeywell', 'Hormel Foods', 'Host Hotels & Resorts', 'Howmet Aerospace', 'HP Inc.', 'Hubbell Incorporated', 'Humana', 'Huntington Bancshares', 'Huntington Ingalls Industries', 'IBM', 'IDEX Corporation', 'Idexx Laboratories', 'Illinois Tool Works', 'Illumina', 'Incyte', 'Ingersoll Rand', 'Insulet', 'Intel', 'Intercontinental Exchange', 'International Flavors & Fragrances', 'International Paper', 'Interpublic Group of Companies (The)', 'Intuit', 'Intuitive Surgical', 'Invesco', 'Invitation Homes', 'IQVIA', 'Iron Mountain', 'J.B. Hunt', 'Jack Henry & Associates', 'Jacobs Solutions', 'Johnson & Johnson', 'Johnson Controls', 'JPMorgan Chase', 'Juniper Networks', "Kellogg's", 'Kenvue', 'Keurig Dr Pepper', 'KeyCorp', 'Keysight', 'Kimberly-Clark', 'Kimco Realty', 'Kinder Morgan', 'KLA Corporation', 'Kraft Heinz', 'Kroger', 'L3Harris', 'LabCorp', 'Lam Research', 'Lamb Weston', 'Las Vegas Sands', 'Leidos', 'Lennar', 'Linde plc', 'Live Nation Entertainment', 'LKQ Corporation', 'Lockheed Martin', 'Loews Corporation', "Lowe's", 'Lululemon Athletica', 'LyondellBasell', 'M&T Bank', 'Marathon Oil', 'Marathon Petroleum', 'MarketAxess', 'Marriott International', 'Marsh McLennan', 'Martin Marietta Materials', 'Masco', 'Mastercard', 'Match Group', 'McCormick & Company', "McDonald's", 'McKesson', 'Medtronic', 'Merck & Co.', 'Meta Platforms', 'MetLife', 'Mettler Toledo', 'MGM Resorts', 'Microchip Technology', 'Micron Technology', 'Microsoft', 'Mid-America Apartment Communities', 'Moderna', 'Mohawk Industries', 'Molina Healthcare', 'Molson Coors Beverage Company', 'Mondelez International', 'Monolithic Power Systems', 'Monster Beverage', "Moody's Corporation", 'Morgan Stanley', 'Mosaic Company (The)', 'Motorola Solutions', 'MSCI', 'Nasdaq, Inc.', 'NetApp', 'Netflix', 'Newmont', 'News Corp (Class A)', 'News Corp (Class B)', 'NextEra Energy', 'Nike, Inc.', 'NiSource', 'Nordson Corporation', 'Norfolk Southern Railway', 'Northern Trust', 'Northrop Grumman', 'Norwegian Cruise Line Holdings', 'NRG Energy', 'Nucor', 'Nvidia', 'NVR, Inc.', 'NXP Semiconductors', "O'Reilly Auto Parts", 'Occidental Petroleum', 'Old Dominion', 'Omnicom Group', 'ON Semiconductor', 'ONEOK', 'Oracle Corporation', 'Otis Worldwide', 'Paccar', 'Packaging Corporation of America', 'Palo Alto Networks', 'Paramount Global', 'Parker Hannifin', 'Paychex', 'Paycom', 'PayPal', 'Pentair', 'PepsiCo', 'Pfizer', 'PG&E Corporation', 'Philip Morris International', 'Phillips 66', 'Pinnacle West', 'Pioneer Natural Resources', 'PNC Financial Services', 'Pool Corporation', 'PPG Industries', 'PPL Corporation', 'Principal Financial Group', 'Procter & Gamble', 'Progressive Corporation', 'Prologis', 'Prudential Financial', 'Public Service Enterprise Group', 'PTC', 'Public Storage', 'PulteGroup', 'Qorvo', 'Quanta Services', 'Qualcomm', 'Quest Diagnostics', 'Ralph Lauren Corporation', 'Raymond James', 'RTX Corporation', 'Realty Income', 'Regency Centers', 'Regeneron', 'Regions Financial Corporation', 'Republic Services', 'ResMed', 'Revvity', 'Robert Half', 'Rockwell Automation', 'Rollins, Inc.', 'Roper Technologies', 'Ross Stores', 'Royal Caribbean Group', 'S&P Global', 'Salesforce', 'SBA Communications', 'Schlumberger', 'Seagate Technology', 'Sealed Air', 'Sempra Energy', 'ServiceNow', 'Sherwin-Williams', 'Simon Property Group', 'Skyworks Solutions', 'J.M. Smucker Company (The)', 'Snap-on', 'SolarEdge', 'Southern Company', 'Southwest Airlines', 'Stanley Black & Decker', 'Starbucks', 'State Street Corporation', 'Steel Dynamics', 'Steris', 'Stryker Corporation', 'Synchrony Financial', 'Synopsys', 'Sysco', 'T-Mobile US', 'T. Rowe Price', 'Take-Two Interactive', 'Tapestry, Inc.', 'Targa Resources', 'Target Corporation', 'TE Connectivity', 'Teledyne Technologies', 'Teleflex', 'Teradyne', 'Tesla, Inc.', 'Texas Instruments', 'Textron', 'Thermo Fisher Scientific', 'TJX Companies', 'Tractor Supply', 'Trane Technologies', 'TransDigm Group', 'Travelers Companies (The)', 'Trimble Inc.', 'Truist', 'Tyler Technologies', 'Tyson Foods', 'U.S. Bank', 'UDR, Inc.', 'Ulta Beauty', 'Union Pacific Corporation', 'United Airlines Holdings', 'United Parcel Service', 'United Rentals', 'UnitedHealth Group', 'Universal Health Services', 'Valero Energy', 'Ventas', 'Veralto', 'Verisign', 'Verisk', 'Verizon', 'Vertex Pharmaceuticals', 'VF Corporation', 'Viatris', 'Vici Properties', 'Visa Inc.', 'Vulcan Materials Company', 'Wabtec', 'Walgreens Boots Alliance', 'Walmart', 'Warner Bros. Discovery', 'Waste Management', 'Waters Corporation', 'WEC Energy Group', 'Wells Fargo', 'Welltower', 'West Pharmaceutical Services', 'Western Digital', 'WestRock', 'Weyerhaeuser', 'Whirlpool Corporation', 'Williams Companies', 'Willis Towers Watson', 'W. W. Grainger', 'Wynn Resorts', 'Xcel Energy', 'Xylem Inc.', 'Yum! Brands', 'Zebra Technologies', 'Zimmer Biomet', 'Zions Bancorporation', 'Zoetis']

In [4]:
def remove_special_chars(input): #takes the string and parses it for 
    bad_chars = [';', ':', '!', "*", "(",")"]
    for i in bad_chars:
        input = input.replace(i, ' ')
    return input

In [5]:
def generate_links(input): #input is the tickers list and output is a list of links to be webscraped
    ret = []
    for i in input:
        ret.append("https://www.marketwatch.com/investing/stock/" + i + "?mod=search_symbol")
    return ret

In [6]:
#descriptions from marketwatch.com
def scrape_descriptions(input): #input is a list of links, output is a dictionary that is like this {ticker: descriptions}
    i = 0
    ret = {}
    for link in input: 
        client = uReq(link) #accesses the webpage
        if (client.getcode() == 200): #checks for fails
            page_html = client.read()
            client.close()
            #parses the html to find the description
            page_soup = soup(page_html, "html.parser")
            text = page_soup.findAll('p', {"class":"description__text"})
            if(len(text) == 0):#for some stocks it won't catch the descriptions, so you need to go back and manually add them
                print("~~~FAIL for: " + tickers[i])
                time.sleep(5)
                i += 1
                ret[tickers[i]] = ""
                continue
            text = text[0].string
            ret[tickers[i]] = text
            print("SUCCESS for: " + tickers[i])
            i += 1
            time.sleep(5) #waits 5 seconds between requests to avoid being overloading the servers
        else:
            print("~~~FAIL for: " + tickers[i])
            i += 1
            client.close()
            ret[tickers[i]] = ""
    return ret

        


In [7]:
def get_keywords(input): #input is a dictionary of stock tickers and their descriptions, output is  a dictionary like this {ticker, [keywords]}
    i = 0 #keeps the index so that I can add the industry and subindustry from the hardcoded lists
    nlp = spacy.load("en_core_web_sm") #loads the language model
    ret = {} #to return
    for ticker in input:
        tmp = [industries[i],subindustries[i], ticker, names[i]]
        doc = nlp(remove_special_chars(input[ticker])) #I remove special characters just in case it messes up the keyword detection since this happened in some tests
        for ent in doc.ents:
            if((ent.label_ == "ORG") | (ent.label_ == "PERSON") | (ent.label_ == "PRODUCT") | (ent.label_ == "NORG")): #checks if the keywords are relevant
                tmp.append(ent.text)
        tmp.sort() #I sorted them to make it faster to check later on.
        ret[ticker] = tmp 
        i += 1
    return ret
        



In [8]:
def descriptions_to_json(t, name): #input is the list of tickers and the name you want the json file to be
    links = generate_links(t)
    descriptions = scrape_descriptions(links)
    keywords = get_keywords(descriptions)
    f = open(name,"w")
    f.write(json.dumps(keywords))
    f.close()

In [9]:
#recursive function that finds the closest item alphabetically to the word that you're given, make sure word is lowercase
#try to avoid using this one, find_closest_lev is slower but should be more accurate
def find_closest_recursive(word, list): #word is the word you want to find the closest alphabetical match to in the list
    # base case
    if(len(list) == 1):
        return list[0]
    if(len(list) == 2):
        if (word > list[0]):
            return list[1]
        else:
            return list[0]
    ##recursive step
    index = int(len(list) / 2)
    tmp = list[index] #tmp is a holder variable so that I can make it lowercase without affecting the main list
    tmp.lower()
    if(tmp == word):
        return word
    if(word > tmp): #word is greater alphabetically than the middle of the list
        return find_closest(word,list[:index]) #recurse with the second half of the list
    else:
        return find_closest(word,list[index:]) #recurse with the first half of the list

In [10]:
#iterative function that returns the word that's the closest according to leventenshtein distance to the wanted word
def find_closest_lev(word,list):
    word.lower() #keep all words in lowercase to make comparing easier, even though it isn't very relevant for lev distances 
    index = 0
    list[0].lower()
    distance = lev(word,list[0])
    for i in range(1,len(list) - 1):
        list[i].lower()
        if(lev(word,list[i]) < distance):
            index = i
            distance = lev(word,list[i])
    return list[index]

In [11]:
#gives a list of tickers that best match the article based off the articles keywords
#input is the articles keywords, stocks is the keywords associated with the stocks(most likely the keywords in ticker_keywords.json), n is the amount of matches you want to recieve(I would stick with around 3) 
def match(input, stocks, n):
    listofzeros = [0] * len(tickers) #just used to initialize the dataframe
    df = pd.DataFrame({"tickers": tickers,"distance":listofzeros}) #the top ten closest stocks with the closest words to the keyword get an increasing to their ranking in order. 
    for word in input:
        #distances = []
        for ticker in stocks:
            # df.loc[df["tickers"].isin([ticker]),'matches'] += 1
            df.loc[df["tickers"].isin([ticker]),"distance"] += lev(word,find_closest_lev(word,stocks[ticker]))
            #distances.append(lev(word,find_closest_lev(word,stocks[ticker]))) #try another version of this function where you just add all the distances and pick the smallest ones
        #df_distances = pd.DataFrame({"tickers":tickers,"distances":distances}) #puts the distances of all the words
        #updates the rankings
        """index = 11
        for row in df_distances.nsmallest(10,"distances"):
            df[df["tickers"] == row["tickers"]]["rating"] += index
            index-=1"""
    return df.nsmallest(n,"distance")["tickers"].to_list()
        


In [27]:
def match_industry(input): #gives a list of all industries and subindustries that are within the tolerance. Input is the articles keywords
    matches = set() #set to avoid duplicates 
    for word in input:
        #can optimize this code by putting the industries and subindustries into a sorted list and then doing binary search
        #checks if matches to industries
        for industry in industries:
            if(lev(word,industry) <= 3):
                matches.add(industry)
        #checks if matches to subindustries
        for subindustry in subindustries:
            if(lev(word,subindustry) <= 3):
                matches.add(subindustry)
    return matches 

In [30]:
#SAMPLE CODE

#uses the match function
f = open("data/ticker_keywords.json","r")
stocks = json.loads(f.read())
f.close()
input = ["Financial", "National generl", "allstate","Utlities"]
output = match(input, stocks, 3)
print("Direct matches:")
print(output)

#uses the match_industry function
industries_list = match_industry(input) #generates a list of possible industries that could be effected by the news story
print("Industries:")
print(industries_list)
f = open("data/industries.json","r")
industries_to_stocks = json.loads(f.read()) #loads the dictionary of what stocks are associated with which industries
final_industries_list = set()
for i in industries_list:
    for index in industries_to_stocks[i]: 
        final_industries_list.add(index)
print("Stocks in industries effected:")
print(final_industries_list)

Direct matches:
['ALL', 'RMD', 'TSN']
Industries:
{'Utilities', 'Financials'}
Stocks in industries effected:
{'PRU', 'COF', 'MS', 'AXP', 'ETR', 'CINF', 'DTE', 'EIX', 'MCO', 'MTB', 'CMA', 'PCG', 'CB', 'DFS', 'PNC', 'AEE', 'SCHW', 'JKHY', 'PPL', 'ED', 'FDS', 'L', 'ALL', 'ZION', 'TROW', 'PGR', 'CNP', 'AFL', 'PFG', 'TFC', 'LNT', 'SYF', 'HBAN', 'TRV', 'PEG', 'CEG', 'BX', 'NI', 'SO', 'BK', 'DUK', 'WTW', 'BLK', 'AES', 'MSCI', 'AON', 'HIG', 'AIG', 'ICE', 'USB', 'CFG', 'AEP', 'FE', 'WRB', 'AWK', 'MA', 'GL', 'AMP', 'SPGI', 'BRK.B', 'ATO', 'CMS', 'NRG', 'AIZ', 'GPN', 'ACGL', 'AJG', 'MMC', 'FLT', 'FI', 'FITB', 'BEN', 'PNW', 'BAC', 'PYPL', 'CME', 'V', 'STT', 'GS', 'C', 'NTRS', 'RF', 'EG', 'RJF', 'IVZ', 'D', 'WFC', 'JPM', 'SRE', 'WEC', 'XEL', 'ES', 'KEY', 'NDAQ', 'EXC', 'MET', 'CBOE', 'MKTX', 'NEE', 'BRO', 'FIS', 'EVRG'}
